In [ ]:
!pip install prophet dagshub scikit-learn pandas numpy mlflow==2.2.2

In [ ]:
import mlflow
import mlflow.prophet
import pandas as pd
import numpy as np
from prophet import Prophet
import os
import dagshub

# Setup MLflow connection (same as experiment notebook)
os.environ['MLFLOW_TRACKING_USERNAME'] = 'nipkha21'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '202fb8a4c58a90b0eb3598b1037498eb6fe9f593'

dagshub.init(repo_owner='TomC333', repo_name='ml-walmart-recruiting', mlflow=True)
mlflow.set_tracking_uri('https://dagshub.com/TomC333/ml-walmart-recruiting.mlflow')

print("MLflow setup completed")

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=e97b781b-1aa4-4bf6-bc5f-edcd2d423800&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=ca490c258b72eaf855fafd04a4abbba3558b8d824222124399499944259164ec




Accessing as nipkha21

Initialized MLflow to track repo "TomC333/ml-walmart-recruiting"

Repository TomC333/ml-walmart-recruiting initialized!

MLflow setup completed


In [ ]:
# Load the registered Prophet model directly
model_name = "Prophet_Sales_Forecasting"
run_id = "62f20e6cd1494a70b73cf0d91d884b19"

print(f"Loading Prophet model from run: {run_id}")

try:
    # Load the Prophet model directly
    loaded_model = mlflow.prophet.load_model(f"runs:/{run_id}/prophet_model")
    print("✅ Successfully loaded Prophet model from MLflow")
except Exception as e:
    print(f"Error loading Prophet model: {e}")
    # Alternative: Load using model registry
    try:
        loaded_model = mlflow.prophet.load_model(f"models:/{model_name}/1")
        print("✅ Successfully loaded Prophet model from Model Registry")
    except Exception as e2:
        print(f"Error loading from registry: {e2}")
        loaded_model = None

# Since preprocessing config wasn't saved, use the known Phase1_Baseline configuration
print("Using Phase1_Baseline configuration (best performing phase)")
preprocessing_config = {
    "min_data_points": 20,
    "train_ratio": 0.8,
    "remove_negative_sales": True,
    "holiday_dates": {
        "superbowl": ['2010-02-12', '2011-02-11', '2012-02-10'],
        "laborday": ['2010-09-10', '2011-09-09', '2012-09-07'],
        "thanksgiving": ['2010-11-26', '2011-11-25', '2012-11-23'],
        "christmas": ['2010-12-31', '2011-12-30', '2012-12-28']
    },
    "external_cols": ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
}

# Phase1_Baseline configuration
best_regressors = []  # No regressors for Phase1_Baseline
holidays_df = None    # No holidays for Phase1_Baseline

print("✅ Configuration set for Phase1_Baseline")
print(f"Regressors: {best_regressors}")
print(f"Holidays: {holidays_df}")

Loading Prophet model from run: 62f20e6cd1494a70b73cf0d91d884b19
✅ Successfully loaded Prophet model from MLflow
Using Phase1_Baseline configuration (best performing phase)
✅ Configuration set for Phase1_Baseline
Regressors: []
Holidays: None


In [ ]:
def create_preprocessing_pipeline(config):
    """Create preprocessing pipeline matching the training process"""

    def preprocess_data(features_data, train_data, stores_data):
        # Merge datasets (exactly as in training)
        df = train_data.merge(features_data, on=['Store', 'Date'], how='inner').merge(stores_data, on=['Store'], how='inner')

        # Handle duplicate columns
        if 'IsHoliday_y' in df.columns:
            df.drop(['IsHoliday_y'], axis=1, inplace=True)
            df.rename(columns={'IsHoliday_x': 'IsHoliday'}, inplace=True)

        # Convert Date and sort
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values(['Store', 'Dept', 'Date'])

        # Remove negative sales if configured
        if config.get("remove_negative_sales", False):
            df = df[df['Weekly_Sales'] >= 0]

        # Create holiday flags (even though Phase1_Baseline doesn't use them, we create them for consistency)
        holiday_dates = config.get("holiday_dates", {})
        df['Is_SuperBowl'] = np.where(df['Date'].isin(pd.to_datetime(holiday_dates.get('superbowl', []))), 1, 0)
        df['Is_LaborDay'] = np.where(df['Date'].isin(pd.to_datetime(holiday_dates.get('laborday', []))), 1, 0)
        df['Is_Thanksgiving'] = np.where(df['Date'].isin(pd.to_datetime(holiday_dates.get('thanksgiving', []))), 1, 0)
        df['Is_Christmas'] = np.where(df['Date'].isin(pd.to_datetime(holiday_dates.get('christmas', []))), 1, 0)

        # Handle missing values in external regressors
        external_cols = config.get("external_cols", [])
        for col in external_cols:
            if col in df.columns:
                df[col] = df.groupby(['Store', 'Dept'])[col].ffill().bfill()

        return df

    return preprocess_data

# Create preprocessing pipeline
preprocess_fn = create_preprocessing_pipeline(preprocessing_config)
print("✅ Preprocessing pipeline created")

✅ Preprocessing pipeline created


In [ ]:
# Load raw data for inference
print("Loading raw data...")
features_data = pd.read_csv('features.csv')
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
stores_data = pd.read_csv('stores.csv')

print(f"Features: {features_data.shape}")
print(f"Train: {train_data.shape}")
print(f"Test: {test_data.shape}")
print(f"Stores: {stores_data.shape}")

# Preprocess training data
processed_train = preprocess_fn(features_data, train_data, stores_data)
print(f"Processed training data: {processed_train.shape}")

# Prepare test data structure
test_df = test_data.merge(features_data, on=['Store', 'Date'], how='left').merge(stores_data, on=['Store'], how='left')
if 'IsHoliday_y' in test_df.columns:
    test_df.drop(['IsHoliday_y'], axis=1, inplace=True)
    test_df.rename(columns={'IsHoliday_x': 'IsHoliday'}, inplace=True)

test_df['Date'] = pd.to_datetime(test_df['Date'])
print(f"Test data prepared: {test_df.shape}")

print("✅ Data loading completed")

Loading raw data...
Features: (8190, 12)
Train: (421570, 5)
Test: (115064, 4)
Stores: (45, 3)
Processed training data: (420285, 20)
Test data prepared: (115064, 15)
✅ Data loading completed


In [ ]:
def prepare_prophet_inference_data(train_df, test_df, store, dept, regressors=None):
    """Prepare Prophet data for a specific store-department combination"""
    # Get store-dept specific training data
    store_dept_train = train_df[(train_df['Store'] == store) & (train_df['Dept'] == dept)].copy()
    store_dept_test = test_df[(test_df['Store'] == store) & (test_df['Dept'] == dept)].copy()

    if len(store_dept_train) < preprocessing_config['min_data_points']:
        return None, None

    # Sort by date
    store_dept_train = store_dept_train.sort_values('Date')
    store_dept_test = store_dept_test.sort_values('Date')

    # Prepare Prophet format - Phase1_Baseline only needs ds and y
    prophet_train = store_dept_train[['Date', 'Weekly_Sales']].copy()
    prophet_train.columns = ['ds', 'y']

    prophet_test = store_dept_test[['Date']].copy()
    prophet_test.columns = ['ds']

    # Phase1_Baseline doesn't use regressors, but keep the structure for consistency
    if regressors:
        for regressor in regressors:
            if regressor in store_dept_train.columns:
                prophet_train[regressor] = store_dept_train[regressor].values
            if regressor in store_dept_test.columns:
                prophet_test[regressor] = store_dept_test[regressor].values

    return prophet_train, prophet_test

def train_and_predict_prophet_phase1(train_data, test_data):
    """Train Prophet model using Phase1_Baseline configuration"""
    try:
        # Initialize Prophet with Phase1_Baseline parameters (no holidays, no regressors)
        model = Prophet(
            daily_seasonality=False,
            weekly_seasonality=True,
            yearly_seasonality=True,
            holidays=None,  # Phase1_Baseline doesn't use holidays
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10.0
        )

        # Phase1_Baseline doesn't add any regressors

        # Fit model
        model.fit(train_data)

        # Make predictions
        forecast = model.predict(test_data)
        return forecast['yhat'].values

    except Exception as e:
        print(f"Error in Prophet prediction: {e}")
        return None

print("✅ Prophet inference functions defined for Phase1_Baseline")

✅ Prophet inference functions defined for Phase1_Baseline


In [ ]:
# Make predictions for all test combinations
print("Starting predictions using Phase1_Baseline configuration...")

# Get unique store-department combinations from test data
test_combinations = test_df[['Store', 'Dept']].drop_duplicates()
print(f"Making predictions for {len(test_combinations)} store-department combinations")

# Initialize predictions dictionary
predictions = {}
successful_predictions = 0
failed_predictions = 0

# Process each combination
for idx, row in test_combinations.iterrows():
    store, dept = row['Store'], row['Dept']

    # Prepare data for this store-dept combination
    prophet_train, prophet_test = prepare_prophet_inference_data(
        processed_train, test_df, store, dept, best_regressors
    )

    if prophet_train is None or prophet_test is None:
        # Use global median as fallback
        global_median = processed_train['Weekly_Sales'].median()
        test_subset = test_df[(test_df['Store'] == store) & (test_df['Dept'] == dept)]
        for _, test_row in test_subset.iterrows():
            key = f"{int(test_row['Store'])}_{int(test_row['Dept'])}_{test_row['Date'].strftime('%Y-%m-%d')}"
            predictions[key] = global_median
        failed_predictions += 1
        continue

    # Make predictions using Phase1_Baseline configuration
    pred_values = train_and_predict_prophet_phase1(prophet_train, prophet_test)

    if pred_values is not None:
        # Store predictions
        test_subset = test_df[(test_df['Store'] == store) & (test_df['Dept'] == dept)].sort_values('Date')
        for i, (_, test_row) in enumerate(test_subset.iterrows()):
            if i < len(pred_values):
                key = f"{int(test_row['Store'])}_{int(test_row['Dept'])}_{test_row['Date'].strftime('%Y-%m-%d')}"
                predictions[key] = max(0, pred_values[i])  # Ensure non-negative
        successful_predictions += 1
    else:
        # Use global median as fallback
        global_median = processed_train['Weekly_Sales'].median()
        test_subset = test_df[(test_df['Store'] == store) & (test_df['Dept'] == dept)]
        for _, test_row in test_subset.iterrows():
            key = f"{int(test_row['Store'])}_{int(test_row['Dept'])}_{test_row['Date'].strftime('%Y-%m-%d')}"
            predictions[key] = global_median
        failed_predictions += 1

    if (successful_predictions + failed_predictions) % 50 == 0:
        print(f"Processed {successful_predictions + failed_predictions} combinations...")

print(f"\nPrediction generation completed:")
print(f"Successful predictions: {successful_predictions}")
print(f"Failed predictions (using fallback): {failed_predictions}")
print(f"Total predictions generated: {len(predictions)}")

DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/_4wi_1b1.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/tmipmcyy.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=99420', 'data', 'file=/tmp/tmpfpw_9l7h/_4wi_1b1.json', 'init=/tmp/tmpfpw_9l7h/tmipmcyy.json', 'output', 'file=/tmp/tmpfpw_9l7h/prophet_model__qrwe6c/prophet_model-20250731182713.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:27:13 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:27:13 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/wkfcr50z.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/ut0ej6fq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

Starting predictions using Phase1_Baseline configuration...
Making predictions for 3169 store-department combinations


18:27:13 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/_0fe1jop.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/1t_4xhgf.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=58582', 'data', 'file=/tmp/tmpfpw_9l7h/_0fe1jop.json', 'init=/tmp/tmpfpw_9l7h/1t_4xhgf.json', 'output', 'file=/tmp/tmpfpw_9l7h/prophet_model8tzpmcob/prophet_model-20250731182714.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:27:14 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:27:14 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/a049hvvs.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/fuwm0loe.json
DEBUG:cmdstanpy:idx

Processed 50 combinations...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/egp8b3wj.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/oenxwmum.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=63483', 'data', 'file=/tmp/tmpfpw_9l7h/egp8b3wj.json', 'init=/tmp/tmpfpw_9l7h/oenxwmum.json', 'output', 'file=/tmp/tmpfpw_9l7h/prophet_modeljcst1zvd/prophet_model-20250731182718.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:27:18 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:27:18 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/foljguw9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/gjou79pp.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/

...

Processed 3100 combinations...


DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/82av1zw4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=59578', 'data', 'file=/tmp/tmpfpw_9l7h/r_ifefv2.json', 'init=/tmp/tmpfpw_9l7h/82av1zw4.json', 'output', 'file=/tmp/tmpfpw_9l7h/prophet_model8s51asmj/prophet_model-20250731183615.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:36:15 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:36:15 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/5ci7davu.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/n0_1lxv_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.b

Processed 3150 combinations...


18:36:24 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/xbysr4r3.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/34zjsj4r.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=58899', 'data', 'file=/tmp/tmpfpw_9l7h/xbysr4r3.json', 'init=/tmp/tmpfpw_9l7h/34zjsj4r.json', 'output', 'file=/tmp/tmpfpw_9l7h/prophet_modelzb2fnakx/prophet_model-20250731183624.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:36:24 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:36:24 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/feebpih8.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpfpw_9l7h/vonksft3.json
DEBUG:cmdstanpy:idx


Prediction generation completed:
Successful predictions: 3037
Failed predictions (using fallback): 132
Total predictions generated: 115064


In [ ]:
# Create submission file
submission_data = []
for key, prediction in predictions.items():
    submission_data.append({
        'Id': key,
        'Weekly_Sales': prediction
    })

submission_df = pd.DataFrame(submission_data)
print(f"Submission dataframe created with {len(submission_df)} rows")

# Prediction statistics
print("Prediction statistics:")
print(f"Mean prediction: {submission_df['Weekly_Sales'].mean():.2f}")
print(f"Median prediction: {submission_df['Weekly_Sales'].median():.2f}")
print(f"Min prediction: {submission_df['Weekly_Sales'].min():.2f}")
print(f"Max prediction: {submission_df['Weekly_Sales'].max():.2f}")

# Save submission file
submission_df.to_csv('prophet_submission.csv', index=False)
print("✅ Submission saved as 'prophet_submission.csv'")

# Display first few rows
print("\nFirst 10 predictions:")
print(submission_df.head(10))

Submission dataframe created with 115064 rows
Prediction statistics:
Mean prediction: 16763.45
Median prediction: 7815.17
Min prediction: 0.00
Max prediction: 382260.43
✅ Submission saved as 'prophet_submission.csv'

First 10 predictions:
               Id  Weekly_Sales
0  1_1_2012-11-02  32925.796897
1  1_1_2012-11-09  27387.048383
2  1_1_2012-11-16  19518.766791
3  1_1_2012-11-23  16606.096193
4  1_1_2012-11-30  22551.899903
5  1_1_2012-12-07  33956.567425
6  1_1_2012-12-14  42751.199814
7  1_1_2012-12-21  42775.539746
8  1_1_2012-12-28  34200.698062
9  1_1_2013-01-04  22499.639963
